In [1]:
!pip3 install easyfsl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 4.3 MB/s eta 0:00:00a 0:00:01


In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import Omniglot
from torchvision.models import resnet18
from tqdm import tqdm

from easyfsl.samplers import TaskSampler
from easyfsl.utils import plot_images, sliding_average

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import certifi
import ssl
from torchvision.datasets.utils import download_and_extract_archive

def download(self) -> None:
    if self._check_integrity():
        print("Files already downloaded and verified")
        return

    filename = self._get_target_folder()
    zip_filename = filename + ".zip"
    url = self.download_url_prefix + "/" + zip_filename

    # Create an SSL context using certifi
    ssl_context = ssl.create_default_context(cafile=certifi.where())

    download_and_extract_archive(url, self.root, filename=zip_filename, md5=self.zips_md5[filename], ssl_context=ssl_context)

image_size = 28

# NB: background=True selects the train set, background=False selects the test set
# It's the nomenclature from the original paper, we just have to deal with it

train_set = Omniglot(
    root="/Users/gaurav/MYProjects/Extion DS Tasks/AutoMl System/Task2/dataset/images_background",
    background=True,
    transform=transforms.Compose(
        [
            transforms.Grayscale(num_output_channels=3),
            transforms.RandomResizedCrop(image_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
        ]
    ),
    download=True,
)
test_set = Omniglot(
    root="/Users/gaurav/MYProjects/Extion DS Tasks/AutoMl System/Task2/dataset/images_background",
    background=False,
    transform=transforms.Compose(
        [
            # Omniglot images have 1 channel, but our model will expect 3-channel images
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize([int(image_size * 1.15), int(image_size * 1.15)]),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),
        ]
    ),
    download=True,
)


100%|██████████| 9464212/9464212 [00:02<00:00, 4146124.35it/s] 


Extracting /Users/gaurav/MYProjects/Extion DS Tasks/AutoMl System/Task2/dataset/images_background/omniglot-py/images_background.zip to /Users/gaurav/MYProjects/Extion DS Tasks/AutoMl System/Task2/dataset/images_background/omniglot-py


100%|██████████| 6462886/6462886 [00:01<00:00, 4114300.05it/s]


Extracting /Users/gaurav/MYProjects/Extion DS Tasks/AutoMl System/Task2/dataset/images_background/omniglot-py/images_evaluation.zip to /Users/gaurav/MYProjects/Extion DS Tasks/AutoMl System/Task2/dataset/images_background/omniglot-py


In [4]:
class PrototypicalNetworks(nn.Module):
    def __init__(self, backbone: nn.Module):
        super(PrototypicalNetworks, self).__init__()
        self.backbone = backbone

    def forward(
        self,
        support_images: torch.Tensor,
        support_labels: torch.Tensor,
        query_images: torch.Tensor,
    ) -> torch.Tensor:
        """
        Predict query labels using labeled support images.
        """
        # Extract the features of support and query images
        z_support = self.backbone.forward(support_images)
        z_query = self.backbone.forward(query_images)

        # Infer the number of different classes from the labels of the support set
        n_way = len(torch.unique(support_labels))
        # Prototype i is the mean of all instances of features corresponding to labels == i
        z_proto = torch.cat(
            [
                z_support[torch.nonzero(support_labels == label)].mean(0)
                for label in range(n_way)
            ]
        )

        # Compute the euclidean distance from queries to prototypes
        dists = torch.cdist(z_query, z_proto)

        # And here is the super complicated operation to transform those distances into classification scores!
        scores = -dists
        return scores


convolutional_network = resnet18(pretrained=True)
convolutional_network.fc = nn.Flatten()
print(convolutional_network)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PrototypicalNetworks(convolutional_network).to(device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [28]:
# Define get_labels function globally
def get_labels(dataset):
    return [instance[1] for instance in dataset._flat_character_images]

# Assign get_labels method to test_set
test_set.get_labels = lambda: get_labels(test_set)

N_WAY = 5 # Number of classes in a task
N_SHOT = 5 # Number of images per class in the support set
N_QUERY = 10 # Number of images per class in the query set
N_EVALUATION_TASKS = 50

# def get_labels(dataset):
#     return [instance[1] for instance in dataset._flat_character_images]

# The sampler needs a dataset with a "get_labels" method. Check the code if you have any doubt!
# test_set.get_labels = lambda: [instance[1] for instance in test_set._flat_character_images]

test_sampler = TaskSampler(
    test_set, n_way=N_WAY, n_shot=N_SHOT, n_query=N_QUERY, n_tasks=N_EVALUATION_TASKS
)

test_loader = DataLoader(
    test_set,
    batch_sampler=test_sampler,
    num_workers=3,
    pin_memory=True,
    collate_fn=test_sampler.episodic_collate_fn,
)


In [32]:




(
    example_support_images,
    example_support_labels,
    example_query_images,
    example_query_labels,
    example_class_ids,
) = next(iter(test_loader))

plot_images(example_support_images, "support images", images_per_row=N_SHOT)
plot_images(example_query_images, "query images", images_per_row=N_QUERY)


PicklingError: Can't pickle <function <lambda> at 0x12f10a200>: attribute lookup <lambda> on __main__ failed

In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import numpy as np
import random
from collections import defaultdict


        
# Create mapping from character labels to image indices
class FewShotDataset(Dataset):
    def __init__(self, data, num_classes, num_examples_per_class):
        self.data = data
        self.num_classes = num_classes
        self.num_examples_per_class = num_examples_per_class
        self.characters = list(set([char for _, char in data]))
        self.class_indices = defaultdict(list)  # Initialize as defaultdict

        for i, (img, char) in enumerate(data):
            self.class_indices[char].append(i)
        
        # Convert characters to numeric labels
        self.char_to_idx = {char: idx for idx, char in enumerate(self.characters)}

    def __len__(self):
        return self.num_classes * self.num_examples_per_class

    def __getitem__(self, index):
        class_idx = index // self.num_examples_per_class
        example_idx = index % self.num_examples_per_class
        
        # Get the character and select a random example from that class
        char = self.characters[class_idx]
        img_idx = random.choice(self.class_indices[char])
        img, _ = self.data[img_idx]  # Get the image, ignore the label

        # Return image and its numeric label
        return img, class_idx

# Neural network encoder for Prototypical Networks (CNN for image data)
class ProtoNet(nn.Module):
    def __init__(self):
        super(ProtoNet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),  # 1 input channel for grayscale
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Linear(64 * 1 * 1, 64)  # Output size for the fully connected layer

    def forward(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1)  # Flatten the feature map
        x = self.fc(x)
        return x

# Function to compute prototypes
def compute_prototypes(embeddings, labels, num_classes):
    prototypes = []
    for class_idx in range(num_classes):
        class_embeddings = embeddings[labels == class_idx]
        prototype = class_embeddings.mean(dim=0)
        prototypes.append(prototype)
    return torch.stack(prototypes)

# Euclidean distance function
def euclidean_distance(x, y):
    return torch.sqrt(((x - y) ** 2).sum(dim=-1))

# Load Omniglot dataset
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

omniglot_train = datasets.Omniglot(root='/Users/gaurav/MYProjects/Extion DS Tasks/AutoMl System/Task2/dataset/images_background/images_background', background=True, download=True, transform=transform)
omniglot_test = datasets.Omniglot(root='/Users/gaurav/MYProjects/Extion DS Tasks/AutoMl System/Task2/dataset/images_background/images_background', background=False, download=True, transform=transform)

# Parameters
num_classes = 5  # Number of classes for few-shot tasks
num_examples_per_class = 5  # Few-shot learning: 5 examples per class
num_query = 10  # Number of query examples per class
num_episodes = 1000  # Number of episodes (tasks) to train on
num_epochs = 20  # Number of training epochs
num_ways = 5  # Number of classes in a few-shot task
num_shot = 5  # Number of support examples per class
num_query = 10  # Number of query examples per class
lr = 0.001  # Learning rate
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create data loaders
train_dataset = FewShotDataset(omniglot_train, num_classes, num_examples_per_class)
train_loader = DataLoader(train_dataset, batch_size=num_classes * num_examples_per_class, shuffle=True)
test_dataset = FewShotDataset(omniglot_test, num_classes, num_examples_per_class)
test_loader = DataLoader(test_dataset, batch_size=num_classes * num_examples_per_class, shuffle=True)

# Initialize ProtoNet and optimizer
model = ProtoNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = model(images)
        prototypes = compute_prototypes(embeddings, labels, num_classes)
        query_indices = torch.tensor([i for i in range(num_classes * num_examples_per_class) if i % num_examples_per_class >= num_shot]).to(device)
        support_indices = torch.tensor([i for i in range(num_classes * num_examples_per_class) if i % num_examples_per_class < num_shot]).to(device)    

        query_embeddings = embeddings[query_indices]
        support_embeddings = embeddings[support_indices]
        query_labels = labels[query_indices] % num_classes
        support_labels = labels[support_indices] % num_classes

        query_prototypes = prototypes[query_labels]
        distances = euclidean_distance(query_embeddings[:, None, :], query_prototypes)
        log_p_y = (-distances).log_softmax(dim=-1)
        loss = -log_p_y.mean()
        

        loss.backward()
        optimizer.step()

    # Evaluate the model
    model.eval()


Files already downloaded and verified
Files already downloaded and verified


KeyboardInterrupt: 